In [12]:
import json
import csv

In [21]:
f = open('ts_data.json')
ts_data = json.load(f)
#print(ts_data['songs'][0]['lyrics'])

In [ ]:
filename = 'data.csv'

In [23]:
ts_data_csv = []
for i in range(len(ts_data['songs'])):
    ts_data_csv = []
    ts_data_csv.append(ts_data['songs'][i]['lyrics'])
    ts_data_csv += [1]
    

['I walked through the door with you, the air was cold\nBut somethin\' \'bout it felt like home somehow\nAnd I left my scarf there at your sister\'s house\nAnd you\'ve still got it in your drawer, even now\n\nOh, your sweet disposition and my wide-eyed gaze\nWe\'re singin\' in the car, getting lost upstate\nAutumn leaves fallin\' down like pieces into place\nAnd I can picture it after all these days\n\nAnd I know it\'s long gone and\nThat magic\'s not here no more\nAnd I might be okay, but I\'m not fine at all\nOh, oh, oh\n\n\'Caus there we ar again on that little town street\nYou almost ran the red \'cause you were lookin\' over at me\nWind in my hair, I was there\nI remember it all too well\n\nPhoto album on the counter, your cheeks were turnin\' red\nYou used to be a little kid with glasses in a twin-sized bed\nAnd your mother\'s tellin\' stories \'bout you on the tee-ball team\nYou taught me \'bout your past, thinkin\' your future was me\nAnd you were tossing me the car keys, "Fuck